In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import random
import torch

In [3]:
with open('./Network dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', newline='') as f:
            reader = csv.reader(f)#속성 85개
            data = list(reader)

In [4]:
col = [2, 4, 5, 7, 8, 9, 10, 11, 12, 70, 71, 72]
target_list = [[0] if data[i][84] == 'BENIGN' else [1] for i in range(len(data))]

y_train = np.array(target_list[1:]).astype(np.float32)
X_train = np.array(data[1:])[:,col].astype(np.float32)
attribute = np.array(data[0])[col] 
#Benign 0, DDos 1
mean = [38257.566, 8879.619, 7.600288, 16241648.0, 4.8749166, 4.5727744,
     939.46344, 5960.4766, 538.53564, 4.5727744, 5960.4766, 4247.437]
std = [23057.252, 19754.605, 3.8815773, 31524304.0, 15.42284,21.755308, 
       3249.3965, 39218.25, 1864.1251, 21.755308, 39218.25, 8037.763]

In [5]:
for i in range(12):
    X_train[:,i] = (X_train[:,i] - mean[i])/std[i]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=321)

In [18]:
import torch.nn as nn
import torch.nn.functional as F

if torch.cuda.is_available():
    device = torch.device("cuda")          # CUDA 장치 객체(device object)로
    print("use gpu")
else:
    print("use cpu")


class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(12, 120)
        #self.fc2 = nn.Linear(120, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 1)
        self.dropout = torch.nn.Dropout()
        
    def forward(self, x):
        x = x.to(device)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.fc4(x)
        x = torch.sigmoid(x)
        
        #x = x.to(device)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc3(x))
        #x = self.fc4(x)
        #x = torch.sigmoid(x) # drop out X 
        return x


net = Net()

use gpu


In [20]:
def init_all(model, init_func, *params, **kwargs):
    for p in model.parameters():
        init_func(p, *params, **kwargs)


#init_all(net, torch.nn.init.normal_, mean=0., std=1)

In [22]:
import torch.optim as optim

criterion = nn.BCELoss()
#criterion = nn.MSELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.0002)

In [23]:
from sklearn.model_selection import KFold
from torch.autograd import Variable

net = net.to(device)

kfold =KFold(n_splits=5)
avg_accuracy = 0
TP = 0
FP = 0
FN = 0
sum_incorrect = 0
for epoch in range(5):
    fold = 0
    for  train_index, test_index in kfold.split(X_train, y_train): 
            x_train_fold = X_train[train_index]
            x_val_fold = X_train[test_index]
            y_train_fold = y_train[train_index]
            y_val_fold = y_train[test_index]
            fold += 1
            #print(x_train_fold.shape)
            for i in range(len(x_train_fold)//100):
                # get the inputs
                inputs = torch.tensor(x_train_fold[100*i:100*i+100])
                labels = torch.tensor(y_train_fold[100*i:100*i+100])
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                outputs = net(inputs)
                #print(outputs)
                #print(labels)
                labels = labels.to(device)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
            running_loss = 0.0
            incorrect = 0
            total = 0
            for i in range(len(x_val_fold)//100):
                # get the inputs
                inputs = torch.tensor(x_val_fold[100*i:100*i+100])
                labels = torch.tensor(y_val_fold[100*i:100*i+100])
                outputs = net(inputs)
                #print(outputs[1])
                labels = labels.to(device)
                loss = criterion(outputs, labels)
                outputs[outputs > 0.5] = 1
                outputs[outputs <= 0.5] = 0
                result = labels - outputs
                result_abs = abs(result)
                FN += ((result+result_abs).sum())/2
                TP += (labels + outputs - result_abs).sum()/2
                total += len(outputs)
                incorrect += result_abs.sum()
                    # print statistics
                running_loss += float(loss.item())
                #print("FN {} FP {} TP {}".format(FN, FP, TP))
            sum_incorrect += incorrect
            accuracy = 100 * (total - incorrect) / total
            avg_accuracy += accuracy
            print('{} epoch {} fold validation loss : {}'.format(epoch+1,fold, running_loss/len(x_val_fold)), end="")
            print(' => Accuracy of the network on the validation: %d %%' % (accuracy))
            running_loss = 0.0
                
FP = (sum_incorrect - FN)
Precision = TP/(TP + FP)
Recall = TP/(TP + FN)
print("--------------------------------------------------------------------------------------")
print('Finished Training')
print("average accuraccy : {}".format(avg_accuracy//25))
print("Precision : {}, Recall : {}, F1-score : {}".format(Precision, Recall, 2*((Precision*Recall)/(Precision+Recall))))

1 epoch 1 fold validation loss : 0.2613063945377168 => Accuracy of the network on the validation: 70 %
1 epoch 2 fold validation loss : 0.07685751358615237 => Accuracy of the network on the validation: 89 %
1 epoch 3 fold validation loss : 0.03510542189365364 => Accuracy of the network on the validation: 94 %
1 epoch 4 fold validation loss : 0.03595601636611427 => Accuracy of the network on the validation: 95 %
1 epoch 5 fold validation loss : 0.16935091527024546 => Accuracy of the network on the validation: 82 %
2 epoch 1 fold validation loss : 0.10451438516522192 => Accuracy of the network on the validation: 88 %
2 epoch 2 fold validation loss : 0.022065627637183137 => Accuracy of the network on the validation: 96 %
2 epoch 3 fold validation loss : 0.013787822944161475 => Accuracy of the network on the validation: 97 %
2 epoch 4 fold validation loss : 0.021226676166789048 => Accuracy of the network on the validation: 97 %
2 epoch 5 fold validation loss : 0.11677459149111181 => Accura

In [ ]:
#with parameter random initialization